In [6]:
# 実験のためには、複数のマイクロホンで収録した音声が必要になる　しかし、それを用意するのはハードルが高い
# そこで、シミュレータを用いて音を生成する
# 単一のwavファイルから生成する？

import wave as wave
import pyroomacoustics as pa
import numpy as np
import scipy.signal as signal

np.random.seed(0)

# 畳み込みに用いる音声波形
clean_wave_file = "./CMU_ARCTIC/cmu_us_aew_arctic/wav/arctic_a0001.wav"


# 単一マイクへの伝搬信号の生成過程を示す（？）
wav = wave.open(clean_wave_file)

"""
getnframes                        ->サンプル数取得
readframes                        ->波形データ読み込み
np.frombuffer                     ->dataを2バイトの数値列に変換（？　何のためにやってるのかしっかり理解はできない）
data = data/np.iinfo(np.int16).max->dataの値を2バイトの変数がとりうる値の最大値で正規化　なるほどね -1~1の間にしてると　←「なるほどね」

#framerate = wav.getframerate()
#print(framerate) 
# 16000だった　以下でsample_rateを手動で定めているが、特に手動で定めることに意味があったわけではないらしい
"""

data = wav.readframes(wav.getnframes())
data = np.frombuffer(data, dtype=np.int16)
data = data/np.iinfo(np.int16).max
wav.close()

sample_rate = 16000

n_impulse_length=512 # 畳み込むインパルス応答長（？　何のこと？）
impulse_response = np.random.normal(size=n_impulse_length) # インパルス応答を乱数で生成(ダミー)　←どういうこと？

conv_data = signal.convolve(data, impulse_response, mode='full') # mode: インパルス応答を畳み込んだ後の出力波形を途中で打ち切るかどうかを決めるパラメータらしい



"""
おそらくこのサンプルコードは、インパルス応答の畳み込みを行う「流れ」のみを示すためのものである。
具体的な「シミュレータ上でのインパルス応答」は次のコードで示されるっぽい（？）。
"""

'\nおそらくこのサンプルコードは、インパルス応答の畳み込みを行う「流れ」のみを示すためのものである。\n具体的な「シミュレータ上でのインパルス応答」は次のコードで示されるっぽい（？）。\n'

In [7]:
import wave as wave
import pyroomacoustics
import numpy as np

np.random.seed(0)

clean_wave_files=["./CMU_ARCTIC/cmu_us_aew_arctic/wav/arctic_a0001.wav"]

n_sources = len(clean_wave_files)

n_samples = 0

for clean_wave_file in clean_wave_files:
    wav = wave.open(clean_wave_file)
    if n_samples < wav.getnframes():
        n_samples = wav.getnframes()
    wav.close()

clean_data = np.zeros([n_sources, n_samples])

s = 0
for clean_wave_file in clean_wave_files:
    wav = wave.open(clean_wave_file)
    data = wav.readframes(wav.getnframes())
    data = np.frombuffer(data, dtype=np.int16)
    data = data/np.iinfo(np.int16).max
    clean_data[s, :wav.getnframes()] = data # ←この表記方法がよくわからない　調べる必要あり
    wav.close()
    s=s+1

# シミュレーションのパラメータ
    
# サンプリング周波数
sample_rate = 16000

# 音声と雑音との比率 [dB]
SNR=90

# 部屋の大きさ
room_dim = np.r_[10.0, 10.0, 10.0]

# マイクロホンアレイを置く部屋の場所
mic_array_loc = room_dim / 2 + np.random.randn(3)*0.1

# マイクロホンアレイのマイクロホン配置
mic_alignments = np.array(
    [
        [-0.01, 0.0, 0.0],
        [0.01, 0.0, 0.0],
    ]
)

# マイクロホン数
n_channels = np.shape(mic_alignments)[0]

# get the microphone array
R = mic_alignments .T+mic_array_loc[:, None]
print(R.T)

# 部屋を生成する
room = pa.ShoeBox(room_dim, fs=sample_rate, max_order=0)

# 用いるマイクロホンアレイの情報を設定する
room.add_microphone_array(pa.MicrophoneArray(R, fs=room.fs))

# 音源の場所
doas = np.array(
    [[np.pi/2., 0],
     [np.pi/2., np.pi/2.]
    ]
)

# 音源とマイクロホンの距離
distance = 1.

source_locations = np.zeros((3, doas.shape[0]), dtype=doas.dtype)
source_locations[0, :] = np.cos(doas[:, 1]) * np.sin(doas[:, 0])
source_locations[1, :] = np.sin(doas[:, 1]) * np.sin(doas[:, 0])
source_locations[2, :] = np.cos(doas[:, 0])
source_locations *= distance
source_locations += mic_array_loc[:, None]

# 各音源をシミュレーションに追加する
for s in range(n_sources):
    clean_data[s] /= np.std(clean_data[s])
    room.add_source(source_locations[:, s], signal=clean_data[s])

# シミュレーションを回す
room.simulate(snr=SNR)

type(room)



[[5.16640523 5.04001572 5.0978738 ]
 [5.18640523 5.04001572 5.0978738 ]]


pyroomacoustics.room.ShoeBox

In [10]:
# それぞれのマイクロホンについてシミュレーションした音声を再生
import scipy.signal as sp
from IPython.display import display, Audio


simulation_data = room.mic_array.signals
# print(type(simulation_data))
# print(simulation_data.shape)
display(Audio(simulation_data[0, :], rate=room.fs))
print(simulation_data.shape)


N = 512
f, t, stft_data = sp.stft(simulation_data, fs=room.fs, window="hann", nperseg=N) # noverlapがnoneのとき、noverlap = nperseg // 2になるらしい

print("f.shape: ", f.shape)
print("t.shape: ", t.shape)
print("stft_data.shape: ", stft_data.shape)

frame_num = 10

stft_data_inv = np.transpose(stft_data, axes=(1, 0, 2)).conj()

set_index = 1

mean_start_index = (set_index-1)*frame_num
mean_end_index = set_index*frame_num
if mean_end_index > stft_data.shape[2]:
    mean_end_index = stft_data.shape[2]

print(np.einsum("mkt,knt->mn", stft_data[:, :, mean_start_index:mean_end_index], stft_data_inv[:, :, mean_start_index:mean_end_index]))


(2, 62210)
f.shape:  (257,)
t.shape:  (245,)
stft_data.shape:  (2, 257, 245)
[[0.00305937+0.j         0.00299448-0.00030183j]
 [0.00299448+0.00030183j 0.00318611+0.j        ]]


In [23]:
import numpy as np

# 配列[a, :b]のテスト
arr = np.array([[1,2,3],[4,5,6],[7,8,9]])
print(arr)

print(arr[0, :2])
print(arr[:2])

room_dim = np.r_[10.0, 10.0, 10.0]
print(room_dim)

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[1 2]
[[1 2 3]
 [4 5 6]]
[10. 10. 10.]
